In [55]:
# Install necessary libraries

# !pip install tensorflow scikit-learn pandas


In [287]:
# import libraries
import pandas as pd

import tensorflow as tf
from sklearn.model_selection                  import train_test_split
from tensorflow.keras.preprocessing.text      import Tokenizer
from tensorflow.keras.preprocessing.sequence  import pad_sequences

In [288]:
# configurations

records = 2000
params  = 1000

In [300]:
# Load the dataset
data = pd.read_csv('dataset/browsing_history_dataset.csv')

data = data[:records]

# Combine the query and past search results into a single text feature
data['combined_text'] = data['query'] + ' ' + data['past_searches']

# # Tokenize the text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['combined_text'])

# print(tokenizer, dir(tokenizer))
# Assuming `data` is a DataFrame with columns 'query', 'likes', and 'priority_percentage'
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(data['query'] + ' ' + data['past_searches'])

# Convert text to sequences
query_sequences = tokenizer.texts_to_sequences(data['query'])
likes_sequences = tokenizer.texts_to_sequences(data['past_searches'])

# Pad sequences
query_padded = pad_sequences(query_sequences, padding='post', maxlen = params)
likes_padded = pad_sequences(likes_sequences, padding='post', maxlen = params)


print(query_padded.shape, likes_padded.shape)
# print(data[['query', 'past_searches', 'priority_score']], query_padded.shape)

(2000, 1000) (2000, 1000)


In [290]:
query_sequences
data['query'][0]
# tokenizer.texts_to_sequences([data['query'][0]])

'Deep tonight indeed war per.'

In [291]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, SimpleRNN, Dense

# Define the model architecture
query_input = Input(shape=(None,), dtype='int32')
likes_input = Input(shape=(None,), dtype='int32')

# Embedding layers for both inputs
query_embedding = Embedding(input_dim=params, output_dim=128)(query_input)
likes_embedding = Embedding(input_dim=params, output_dim=128)(likes_input)

# Concatenate the embeddings
merged = Concatenate()([query_embedding, likes_embedding])

# Simple RNN layer
rnn_layer = SimpleRNN(128)(merged)

# Dense layer for regression
output = Dense(1)(rnn_layer)

# Compile the model
model = Model(inputs=[query_input, likes_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
print(query_padded.shape, likes_padded.shape)

(2000, 1000) (2000, 1000)


In [292]:
# model.fit(X_train, y_train, epochs=10, batch_size=32)
model.fit([query_padded, likes_padded], data['priority_score'], epochs=2, batch_size=32)


Epoch 1/2
63/63 ━━━━━━━━━━━━━━━━━━━━ 19s 287ms/step - accuracy: 0.0047 - loss: 0.1148
Epoch 2/2
63/63 ━━━━━━━━━━━━━━━━━━━━ 18s 283ms/step - accuracy: 0.0036 - loss: 0.0956


In [293]:
import pickle

# saving
with open('../models/tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

# loading
with open('../models/tokenizer.pickle', 'rb') as handle:
    tokenizer1 = pickle.load(handle)

In [294]:
tokenizer.texts_to_sequences(['x ray physics indeed'])

[[1, 1, 1, 566]]

In [295]:
model.predict([query_padded, likes_padded])

63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 88ms/step


array([[0.418385  ],
       [0.41838494],
       [0.41838503],
       ...,
       [0.41838482],
       [0.41838533],
       [0.41838485]], dtype=float32)

In [242]:
model.save('hello.keras')

In [221]:
tokenizer1.word_index

{'<OOV>': 1,
 'bag': 2,
 'home': 3,
 'air': 4,
 'drop': 5,
 'or': 6,
 'east': 7,
 'stay': 8,
 'lot': 9,
 'civil': 10,
 'fall': 11,
 'like': 12,
 'at': 13,
 'real': 14,
 'his': 15,
 'oil': 16,
 'five': 17,
 'rest': 18,
 'now': 19,
 'center': 20,
 'tell': 21,
 'idea': 22,
 'a': 23,
 'alone': 24,
 'cup': 25,
 'then': 26,
 'join': 27,
 'four': 28,
 'miss': 29,
 'tree': 30,
 'want': 31,
 'accept': 32,
 'race': 33,
 'order': 34,
 'too': 35,
 'kid': 36,
 'all': 37,
 'road': 38,
 'man': 39,
 'set': 40,
 'task': 41,
 'whom': 42,
 'that': 43,
 'describe': 44,
 'push': 45,
 'last': 46,
 'list': 47,
 'ten': 48,
 'agree': 49,
 'short': 50,
 'break': 51,
 'game': 52,
 'hold': 53,
 'yet': 54,
 'have': 55,
 'admit': 56,
 'focus': 57,
 'if': 58,
 'sense': 59,
 'go': 60,
 'win': 61,
 'let': 62,
 'way': 63,
 'next': 64,
 'south': 65,
 'really': 66,
 'try': 67,
 'free': 68,
 'reach': 69,
 'wrong': 70,
 'unit': 71,
 'few': 72,
 'on': 73,
 'my': 74,
 'wish': 75,
 'pm': 76,
 'speech': 77,
 'show': 78,
 'of':

In [298]:
import keras
keras.saving.save_model(model, '../models/model_v1.keras', overwrite=True)
# loaded_model = keras.saving.load_model("model1.keras")

In [255]:
loaded_model.predict([query_padded, likes_padded])

63/63 ━━━━━━━━━━━━━━━━━━━━ 6s 92ms/step


array([[0.53092307],
       [0.53092664],
       [0.5309242 ],
       ...,
       [0.53092647],
       [0.5309263 ],
       [0.5309273 ]], dtype=float32)

In [144]:
for i in range(4):
    q = data['query'][i]
    print(q)
    seq = tokenizer.texts_to_sequences(q)
    pad = pad_sequences(seq)
    print(len(seq), len(pad))
    

Deep tonight indeed war per.
28 28
Wonder common.
14 14
Anyone people cell fill.
24 24
Collection theory beat.
23 23


In [186]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Example data
queries = ["best smartphones 2024", "latest fashion trends"]
likes = ["smartphones, technology, gadgets", "fashion, style, clothing"]
labels = [1, 0] # 1 for relevant, 0 for not relevant

# Tokenize the text
tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(queries + likes)

# Convert text to sequences
queries_seq = tokenizer.texts_to_sequences(queries, maxlen = params)
likes_seq = tokenizer.texts_to_sequences(likes, maxlen = params)

# Pad sequences
queries_seq = pad_sequences(queries_seq, padding='post')
likes_seq = pad_sequences(likes_seq, padding='post')

# Convert sequences to tensors
queries_tensor = tf.convert_to_tensor(queries_seq)
likes_tensor = tf.convert_to_tensor(likes_seq)
labels_tensor = tf.convert_to_tensor(labels)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense

model = Sequential([
    Embedding(input_dim=10000, output_dim=128, input_length=queries_seq.shape[1]),
    SimpleRNN(128),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


# Example: Predicting the priority score of a new search query
new_search_query = "best smartphones 2024"
new_past_searches = "smartphones, technology"

# Preprocess the new data
new_combined_text = new_search_query + ' ' + new_past_searches
new_sequence = tokenizer.texts_to_sequences([new_combined_text])
new_padded_sequence = pad_sequences(new_sequence, padding='post')

# Convert to tensor
new_X = tf.convert_to_tensor(new_padded_sequence)

# Predict
predicted_priority = model.predict(new_X)

# Convert back to the original priority score range
predicted_priority = (predicted_priority * (data['priority_score'].max() - data['priority_score'].min())) + data['priority_score'].min()

print(f"Predicted priority score: {predicted_priority[0][0]}")



NameError: name 'params' is not defined

In [194]:
query_padded.shape

(30, 1000)

In [225]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, SimpleRNN, Dense

# Define the model architecture
query_input = Input(shape=(None,), dtype='int32')
likes_input = Input(shape=(None,), dtype='int32')

# Embedding layers for both inputs
query_embedding = Embedding(input_dim=params, output_dim=128)(query_input)
likes_embedding = Embedding(input_dim=params, output_dim=128)(likes_input)

# Concatenate the embeddings
merged = Concatenate()([query_embedding, likes_embedding])

# Simple RNN layer
rnn_layer = SimpleRNN(128)(merged)

# Dense layer for regression
output = Dense(1)(rnn_layer)

# Compile the model
model = Model(inputs=[query_input, likes_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
print(query_padded.shape, likes_padded.shape)

(1000, 1000) (1000, 1000)


In [ ]:
# model.fit(X_train, y_train, epochs=10, batch_size=32)

model.fit([query_padded, likes_padded], data['priority_score'], epochs=20, batch_size=32)


NameError: name 'tokenizer' is not defined

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer


tokenizer = Tokenizer(num_words=10000, oov_token="<OOV>")
tokenizer.fit_on_texts(['sleep', 'hello', 'good'])



In [41]:
tokenizer.texts_to_sequences(['good'])
tokenizer.to_json()

'{"class_name": "Tokenizer", "config": {"num_words": 10000, "filters": "!\\"#$%&()*+,-./:;<=>?@[\\\\]^_`{|}~\\t\\n", "lower": true, "split": " ", "char_level": false, "oov_token": "<OOV>", "document_count": 3, "word_counts": "{\\"sleep\\": 1, \\"hello\\": 1, \\"good\\": 1}", "word_docs": "{\\"sleep\\": 1, \\"hello\\": 1, \\"good\\": 1}", "index_docs": "{\\"2\\": 1, \\"3\\": 1, \\"4\\": 1}", "index_word": "{\\"1\\": \\"<OOV>\\", \\"2\\": \\"sleep\\", \\"3\\": \\"hello\\", \\"4\\": \\"good\\"}", "word_index": "{\\"<OOV>\\": 1, \\"sleep\\": 2, \\"hello\\": 3, \\"good\\": 4}"}}'

In [44]:
tokenizer.texts_to_sequences(['better', 'sleeping'])

[[1], [1]]

In [53]:
# !pip install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

def preprocess_search_query(query):
    # Tokenize the query
    tokens = word_tokenize(query.lower())  # Convert to lowercase
    
    # Remove punctuation
    tokens = [token for token in tokens if token not in string.punctuation]
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    tokens = [token for token in tokens if token not in stop_words]
    
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    return tokens

# Example search query
search_query = "Machine learning techniques for data analysis"

# Preprocess the search query
preprocessed_query = preprocess_search_query(search_query)
print("Preprocessed tokens:", preprocessed_query)


LookupError: 
**********************************************************************
  Resource [93mpunkt[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt/PY3/english.pickle[0m

  Searched in:
    - '/Users/konda.sandeep/nltk_data'
    - '/Users/konda.sandeep/Projects/personal/major project/backend/env/nltk_data'
    - '/Users/konda.sandeep/Projects/personal/major project/backend/env/share/nltk_data'
    - '/Users/konda.sandeep/Projects/personal/major project/backend/env/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
    - ''
**********************************************************************


In [54]:
import tensorflow as tf

# Define the text containing the words
texts = ["sleeping", "sleep"]

# Initialize Tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()

# Fit tokenizer on texts
tokenizer.fit_on_texts(texts)

# Tokenize the texts
sequences = tokenizer.texts_to_sequences(texts)

# Check vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary size:", vocab_size)

# Print the tokenized sequences
print("Tokenized sequences:", sequences)


Vocabulary size: 3
Tokenized sequences: [[1], [2]]
